In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,351 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Pac

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-09-27 22:37:34--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.98MB/s    in 0.2s    

2021-09-27 22:37:34 (4.98 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Electronics_v1_00.tsv.gz"),sep="\t", header=True, inferSchema=True, timestampFormat="mm/dd/yy" )
df.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+-------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|marketplace|customer_id|review_id     |product_id|product_parent|product_title                                                                                                                                                |product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline                  

In [5]:
# get the shape of the dataframe

print((df.count(), len(df.columns)))

(3093869, 15)


In [6]:
# dropping null values
dropna_df = df.dropna()

dropna_df.count()

3093660

In [7]:
# dropping duplicate rows
electronics_df = dropna_df.dropDuplicates()

electronics_df.count()

3093660

In [8]:
electronics_df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [11]:
from pyspark.sql.functions import *
electronics_df.select(col('review_date'),to_date(col('review_date'),"MM-dd-yyyy").alias("date"))
electronics_df.show()

+-----------+----+
|review_date|date|
+-----------+----+
| 2013-10-14|null|
| 2015-06-30|null|
| 2014-06-19|null|
| 2012-12-24|null|
| 2011-10-03|null|
| 2010-08-30|null|
| 2011-07-12|null|
| 2014-02-27|null|
| 2013-12-09|null|
| 2015-05-28|null|
| 2008-09-25|null|
| 2015-07-14|null|
| 2012-02-08|null|
| 2013-02-18|null|
| 2015-08-30|null|
| 2012-11-15|null|
| 2015-08-22|null|
| 2014-12-06|null|
| 2003-06-01|null|
| 2009-10-31|null|
+-----------+----+
only showing top 20 rows



In [9]:
review_id_table_df = df['review_id', 'customer_id', 'product_id', 'product_parent', 'review_data']
review_id_table_df.show()

AnalysisException: ignored